In [1]:
# coding: utf-8
"""
楽天レビューでトピックを抽出するための学習ファイルの作成
UM用に1文ずつ形態素解析を行っていく
"""
import MeCab
import csv
import collections
import pickle

def parsing(sentence):
    mecab = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if (features[0] == "名詞" and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or features[0] == "形容詞":
            if features[6] == "*":
                list_words.append(res.surface)
            else:
                list_words.append(features[6])
        res = res.next
    return list_words

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def readtsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f, delimiter='\t')
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def writedump(arr, path):
    f = open(path, "w")
    pickle.dump(arr, f)
    f.close()

### コーパス作成
1. 空白行を削除

In [3]:
list_sentences = readtsv("./files/rakuten_corpus/annotation01_tsukuba_corpus_20140930.tsv")
# 同じユーザーの口コミを1文にまとめる
list_sentences_rev = []
for row in list_sentences:
    if row[5] != "":
        list_sentences_rev.append(row[5])


# 形態素解析したリストを作成
list_words = [parsing(row) for row in list_sentences_rev]

# 各単語の頻度を数え上げる
list_words_collection = []
for row in list_words:
    list_words_collection.extend(row)
list_words_collection = collections.Counter(list_words_collection).items()

# 除去語リストを作成
list_words_remove = []
for row in list_words_collection:
    if row[1] < 3 or row[1] > 300:
        list_words_remove.append(row[0])

# 除去語リスト内の単語を削除
list_words_rev = []
for row in list_words:
    list_words_tmp = []
    for word in row:
        if word in list_words_remove:
            pass
        else:
            list_words_tmp.append(word)
    list_words_rev.append(list_words_tmp)

In [6]:
list_words_rev = [[" ".join(row)] for row in list_words_rev]

### 空白でsepしてtxt形式で保存

In [7]:
writecsv(list_words_rev, "./files/rakuten_corpus/rakuten_corpus_for_UM.txt")